In [1]:
import time
import ray
import pandas as pd
import pickle
import winsound
import numpy as np
import genetic_algorithm as ga

In [2]:
run_num = 1
res_file_name = 'tuning.pickle'

In [3]:
col_names = ['#','Dataset', 'Run', 'Pop Size', 'K', 'Fitness Func', 'Crossover Type', 'Crossover Prob', 'Elitism Ratio', \
            'Max Parent Allow', 'Mutation Prob', 'Mutation Rate', 'Max Iter', 'Individual Size','Solution Costs', 'Best Solution', \
            'Stopping Cond', 'Run Time','Min Cost']

In [4]:
res_df = pd.DataFrame(columns=col_names)
res_df.to_pickle(res_file_name)

In [5]:
import ray
ray.init()

2021-01-24 22:36:14,475	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.0.11',
 'raylet_ip_address': '192.168.0.11',
 'redis_address': '192.168.0.11:6379',
 'object_store_address': 'tcp://127.0.0.1:62183',
 'raylet_socket_name': 'tcp://127.0.0.1:52563',
 'webui_url': '127.0.0.1:8265',
 'session_dir': 'C:\\Users\\ISMAIL~1\\AppData\\Local\\Temp\\ray\\session_2021-01-24_22-36-13_322282_12240',
 'metrics_export_port': 54712,
 'node_id': '9a1d86260e8ca502e2161f87a16f7eadb334890a'}

### tuning parameters

In [6]:
population_size = 50

# crossover parameters
crossover_probability = 1.0
crossover_type = 2 
fitness_function = 'cost'

# selection parameters
elitism_ratio_list = [0.2, 0.4]
max_parent_allowance = 0.3

#mutation parameters
mutation_probability = 0.2
mutation_rate_list = [0.1, 0.3, 0.5]

# algorithm settings
data_sets = ['120_1', '120_2', '240_1', '240_2', '360_1', '360_2']
max_iter = 500 
k = 10 
stop_thrs = 10
mutation_level = 0.2

run_num = 3

In [7]:
params_pool = []

for elitism_ratio in elitism_ratio_list:
    for mutation_rate in mutation_rate_list:
            for data in data_sets:
                for run in range(1, run_num+1):

                    individual_size = int(data[0:3])

                    parameters = [
                    data, run, \
                    population_size, k,\
                    fitness_function,  crossover_type, crossover_probability, \
                    elitism_ratio, max_parent_allowance, \
                    mutation_probability, mutation_rate, max_iter, individual_size,\
                    stop_thrs, mutation_level \
                    ]

                    params_pool.append(parameters)

In [8]:
len(params_pool)

108

In [9]:
res_df = pd.read_pickle(res_file_name)
len(res_df)

0

In [10]:
idx = 0

In [11]:
@ray.remote
def run(parameters):
    global idx
    idx += 1 
    try:
        res = ga.run(parameters)
    except:
        print('An error occuered with ', parameters)
        return {}
    print('run',idx, parameters, ' - ',res['run_time'])
    return res

In [12]:
start = time.time()
futures = [run.remote(params) for params in params_pool]
results = ray.get(futures)
end =time.time()
print(end-start)

(pid=15848) run 1 ['120_1', 1, 50, 10, 'cost', 2, 1.0, 0.2, 0.3, 0.2, 0.1, 500, 120, 10, 0.2]  -  8.411845219135284
(pid=20944) run 1 ['120_1', 2, 50, 10, 'cost', 2, 1.0, 0.2, 0.3, 0.2, 0.1, 500, 120, 10, 0.2]  -  8.486831072966257
(pid=17252) run 1 ['120_2', 3, 50, 10, 'cost', 2, 1.0, 0.2, 0.3, 0.2, 0.1, 500, 120, 10, 0.2]  -  8.527751898765564
(pid=16684) run 1 ['120_1', 2, 50, 10, 'cost', 2, 1.0, 0.2, 0.3, 0.2, 0.3, 500, 120, 10, 0.2]  -  8.558575451374054
(pid=11344) run 1 ['120_2', 2, 50, 10, 'cost', 2, 1.0, 0.2, 0.3, 0.2, 0.1, 500, 120, 10, 0.2]  -  8.717179024219513
(pid=2664) run 1 ['120_1', 3, 50, 10, 'cost', 2, 1.0, 0.2, 0.3, 0.2, 0.1, 500, 120, 10, 0.2]  -  8.74061759710312
(pid=4428) run 1 ['120_1', 3, 50, 10, 'cost', 2, 1.0, 0.2, 0.3, 0.2, 0.3, 500, 120, 10, 0.2]  -  8.76799058119456
(pid=23280) run 1 ['120_2', 1, 50, 10, 'cost', 2, 1.0, 0.2, 0.3, 0.2, 0.1, 500, 120, 10, 0.2]  -  8.884107013543447
(pid=5672) run 1 ['120_1', 1, 50, 10, 'cost', 2, 1.0, 0.2, 0.3, 0.2, 0.3, 50

(pid=21312) run 2 ['240_2', 2, 50, 10, 'cost', 2, 1.0, 0.4, 0.3, 0.2, 0.1, 500, 240, 10, 0.2]  -  10.564010214805602
(pid=23468) run 2 ['240_2', 3, 50, 10, 'cost', 2, 1.0, 0.4, 0.3, 0.2, 0.1, 500, 240, 10, 0.2]  -  10.60582411289215
(pid=23288) run 2 ['360_1', 2, 50, 10, 'cost', 2, 1.0, 0.2, 0.3, 0.2, 0.5, 500, 360, 10, 0.2]  -  21.445163337389626
(pid=5940) run 2 ['360_1', 2, 50, 10, 'cost', 2, 1.0, 0.4, 0.3, 0.2, 0.1, 500, 360, 10, 0.2]  -  14.786512017250061
(pid=4552) run 2 ['360_1', 3, 50, 10, 'cost', 2, 1.0, 0.2, 0.3, 0.2, 0.5, 500, 360, 10, 0.2]  -  21.516745988527934
(pid=8784) run 2 ['360_1', 1, 50, 10, 'cost', 2, 1.0, 0.4, 0.3, 0.2, 0.1, 500, 360, 10, 0.2]  -  15.15508648554484
(pid=7388) run 2 ['360_2', 1, 50, 10, 'cost', 2, 1.0, 0.2, 0.3, 0.2, 0.5, 500, 360, 10, 0.2]  -  21.755382585525513
(pid=9864) run 4 ['120_1', 2, 50, 10, 'cost', 2, 1.0, 0.4, 0.3, 0.2, 0.5, 500, 120, 10, 0.2]  -  6.338203183809916
(pid=21884) run 3 ['120_1', 1, 50, 10, 'cost', 2, 1.0, 0.4, 0.3, 0.2, 0.

### results

In [14]:
index = len(res_df)

for params, res in zip(params_pool, results):
    
    if len(res)==0:

        solns = [index] + params[0:13] + ['']*5
    else:    
        solns = [index] + params[0:13] + [res['cost_func'], res['best_soln'], \
                                      res['stop_cond'], res['run_time'], res['min_cost']]
        
    res_df = res_df.append(pd.DataFrame([solns], columns=col_names),ignore_index=True)
    
    index +=1
    
res_df.to_pickle(res_file_name)

In [15]:
len(res_df)

108

In [19]:
dd = res_df.groupby('Dataset').to_frame()
dd

AttributeError: 'DataFrameGroupBy' object has no attribute 'to_frame'

In [ ]:
# beep
winsound.Beep(500, 300)

params = [
population_size, k,\
fitness_function,  crossover_type, crossover_probability, \
elitism_ratio, max_parent_allowance, \
mutation_probability, mutation_rate, max_iter, \
individual_size, stop_thrs, mutation_level \
]